# Workshop Azure Databricks
## 04. Clustering
<img src="https://raw.githubusercontent.com/retkowsky/images/master/AzureDatabricksLogo.jpg"><br>

# Documentation
Présentation https://azure.microsoft.com/fr-fr/services/databricks/

Documentation Azure Databricks : https://docs.microsoft.com/fr-fr/azure/databricks/

Documentation Azure ML : https://docs.microsoft.com/en-us/azure/machine-learning/

Github : https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/azure-databricks

## Clustering
In this exercise, you will use K-Means clustering to segment customer data into five clusters.

### Import the Libraries
You will use the **KMeans** class to create your model. This will require a vector of features, so you will also use the **VectorAssembler** class.

In [0]:
import datetime
now = datetime.datetime.now()
print(now)

2021-02-10 13:55:07.057197

In [0]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

### Load Source Data
The source data for your clusters is in a comma-separated values (CSV) file, and incldues the following features:
- CustomerName: The customer's name
- Age: The customer's age in years
- MaritalStatus: The custtomer's marital status (1=Married, 0 = Unmarried)
- IncomeRange: The top-level for the customer's income range (for example, a value of 25,000 means the customer earns up to 25,000)
- Gender: A numeric value indicating gender (1 = female, 2 = male)
- TotalChildren: The total number of children the customer has
- ChildrenAtHome: The number of children the customer has living at home.
- Education: A numeric value indicating the highest level of education the customer has attained (1=Started High School to 5=Post-Graduate Degree
- Occupation: A numeric value indicating the type of occupation of the customer (0=Unskilled manual work to 5=Professional)
- HomeOwner: A numeric code to indicate home-ownership (1 - home owner, 0 = not a home owner)
- Cars: The number of cars owned by the customer.

> Importer le fichier CSV dans DBFS

In [0]:
file_location = "/FileStore/tables/customers.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
customers = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(customers)

CustomerName,Age,MaritalStatus,IncomeRange,Gender,TotalChildren,ChildrenAtHome,Education,Occupation,HomeOwner,Cars
Aaron Adams,42,0,50000,0,0,0,3,2,1,1
Aaron Alexander,40,1,50000,0,0,0,2,2,1,2
Aaron Allen,63,0,25000,0,2,1,2,1,1,2
Aaron Baker,56,1,50000,0,4,2,2,2,1,2
Aaron Bryant,72,0,75000,0,4,0,4,4,1,2
Aaron Butler,42,1,75000,0,0,0,3,5,1,2
Aaron Campbell,49,0,75000,0,0,0,5,5,1,1
Aaron Carter,42,0,50000,0,0,0,3,2,0,1
Aaron Chen,57,1,75000,0,4,3,4,5,1,0
Aaron Coleman,42,0,50000,0,0,0,3,2,1,1


In [0]:
train_rows = customers.count()

In [0]:
print(train_rows)

18508

In [0]:
customers.describe("Age").show()

+-------+-----------------+
summary| Age|
+-------+-----------------+
 count| 18508|
 mean|54.42289820618111|
 stddev| 11.2494447327056|
 min| 36|
 max| 106|
+-------+-----------------+

### Create the K-Means Model
You will use the feaures in the customer data to create a Kn-Means model with a k value of 5. This will be used to generate 5 clusters.

In [0]:
assembler = VectorAssembler(inputCols = ["Age", "MaritalStatus", "IncomeRange", "Gender", "TotalChildren", "ChildrenAtHome", "Education", "Occupation", "HomeOwner", "Cars"], outputCol="features")
train = assembler.transform(customers)

kmeans = KMeans(featuresCol=assembler.getOutputCol(), predictionCol="cluster", k=5, seed=0)

model = kmeans.fit(train)

print ("OK")

OK

### Get the Cluster Centers
The cluster centers are indicated as vector coordinates.

In [0]:
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[5.31013005e+01 4.17180014e-01 2.50000000e+04 4.80492813e-01
 1.41512663e+00 6.08487337e-01 2.31622177e+00 1.45448323e+00
 5.93086927e-01 1.11464750e+00]
[5.53417813e+01 5.72411296e-01 1.00000000e+05 4.97103548e-01
 2.54380883e+00 1.54272266e+00 3.46198407e+00 4.19116582e+00
 7.16509776e-01 1.94532947e+00]
[5.19737441e+01 5.26868545e-01 5.00000000e+04 4.93961141e-01
 1.34552774e+00 4.98337126e-01 3.23035183e+00 2.77927534e+00
 6.62699107e-01 1.14615789e+00]
[5.60711289e+01 5.83804487e-01 7.50000000e+04 5.03921211e-01
 2.17308043e+00 8.16706183e-01 3.73244574e+00 3.92759438e+00
 7.23326646e-01 1.38063104e+00]
[5.82794840e+01 6.22850123e-01 1.50000000e+05 4.79729730e-01
 2.07248157e+00 3.20638821e+00 3.41461916e+00 4.34705160e+00
 6.48648649e-01 3.10995086e+00]

### Predict Clusters
Now that you have trained the model, you can use it to segemnt the customer data into 5 clusters and show each customer with their allocated cluster.

In [0]:
classif = model.transform(train)
classif.groupBy("cluster").count().orderBy("cluster").show()

+-------+-----+
cluster|count|
+-------+-----+
 0| 2922|
 1| 2762|
 2| 5713|
 3| 5483|
 4| 1628|
+-------+-----+

In [0]:
display(classif)

CustomerName,Age,MaritalStatus,IncomeRange,Gender,TotalChildren,ChildrenAtHome,Education,Occupation,HomeOwner,Cars,features,cluster
Aaron Adams,42,0,50000,0,0,0,3,2,1,1,"Map(vectorType -> dense, length -> 10, values -> List(42.0, 0.0, 50000.0, 0.0, 0.0, 0.0, 3.0, 2.0, 1.0, 1.0))",2
Aaron Alexander,40,1,50000,0,0,0,2,2,1,2,"Map(vectorType -> dense, length -> 10, values -> List(40.0, 1.0, 50000.0, 0.0, 0.0, 0.0, 2.0, 2.0, 1.0, 2.0))",2
Aaron Allen,63,0,25000,0,2,1,2,1,1,2,"Map(vectorType -> dense, length -> 10, values -> List(63.0, 0.0, 25000.0, 0.0, 2.0, 1.0, 2.0, 1.0, 1.0, 2.0))",0
Aaron Baker,56,1,50000,0,4,2,2,2,1,2,"Map(vectorType -> dense, length -> 10, values -> List(56.0, 1.0, 50000.0, 0.0, 4.0, 2.0, 2.0, 2.0, 1.0, 2.0))",2
Aaron Bryant,72,0,75000,0,4,0,4,4,1,2,"Map(vectorType -> dense, length -> 10, values -> List(72.0, 0.0, 75000.0, 0.0, 4.0, 0.0, 4.0, 4.0, 1.0, 2.0))",3
Aaron Butler,42,1,75000,0,0,0,3,5,1,2,"Map(vectorType -> dense, length -> 10, values -> List(42.0, 1.0, 75000.0, 0.0, 0.0, 0.0, 3.0, 5.0, 1.0, 2.0))",3
Aaron Campbell,49,0,75000,0,0,0,5,5,1,1,"Map(vectorType -> dense, length -> 10, values -> List(49.0, 0.0, 75000.0, 0.0, 0.0, 0.0, 5.0, 5.0, 1.0, 1.0))",3
Aaron Carter,42,0,50000,0,0,0,3,2,0,1,"Map(vectorType -> sparse, length -> 10, indices -> List(0, 2, 6, 7, 9), values -> List(42.0, 50000.0, 3.0, 2.0, 1.0))",2
Aaron Chen,57,1,75000,0,4,3,4,5,1,0,"Map(vectorType -> dense, length -> 10, values -> List(57.0, 1.0, 75000.0, 0.0, 4.0, 3.0, 4.0, 5.0, 1.0, 0.0))",3
Aaron Coleman,42,0,50000,0,0,0,3,2,1,1,"Map(vectorType -> dense, length -> 10, values -> List(42.0, 0.0, 50000.0, 0.0, 0.0, 0.0, 3.0, 2.0, 1.0, 1.0))",2


In [0]:
display(classif)

CustomerName,Age,MaritalStatus,IncomeRange,Gender,TotalChildren,ChildrenAtHome,Education,Occupation,HomeOwner,Cars,features,cluster
Aaron Adams,42,0,50000,0,0,0,3,2,1,1,"Map(vectorType -> dense, length -> 10, values -> List(42.0, 0.0, 50000.0, 0.0, 0.0, 0.0, 3.0, 2.0, 1.0, 1.0))",2
Aaron Alexander,40,1,50000,0,0,0,2,2,1,2,"Map(vectorType -> dense, length -> 10, values -> List(40.0, 1.0, 50000.0, 0.0, 0.0, 0.0, 2.0, 2.0, 1.0, 2.0))",2
Aaron Allen,63,0,25000,0,2,1,2,1,1,2,"Map(vectorType -> dense, length -> 10, values -> List(63.0, 0.0, 25000.0, 0.0, 2.0, 1.0, 2.0, 1.0, 1.0, 2.0))",0
Aaron Baker,56,1,50000,0,4,2,2,2,1,2,"Map(vectorType -> dense, length -> 10, values -> List(56.0, 1.0, 50000.0, 0.0, 4.0, 2.0, 2.0, 2.0, 1.0, 2.0))",2
Aaron Bryant,72,0,75000,0,4,0,4,4,1,2,"Map(vectorType -> dense, length -> 10, values -> List(72.0, 0.0, 75000.0, 0.0, 4.0, 0.0, 4.0, 4.0, 1.0, 2.0))",3
Aaron Butler,42,1,75000,0,0,0,3,5,1,2,"Map(vectorType -> dense, length -> 10, values -> List(42.0, 1.0, 75000.0, 0.0, 0.0, 0.0, 3.0, 5.0, 1.0, 2.0))",3
Aaron Campbell,49,0,75000,0,0,0,5,5,1,1,"Map(vectorType -> dense, length -> 10, values -> List(49.0, 0.0, 75000.0, 0.0, 0.0, 0.0, 5.0, 5.0, 1.0, 1.0))",3
Aaron Carter,42,0,50000,0,0,0,3,2,0,1,"Map(vectorType -> sparse, length -> 10, indices -> List(0, 2, 6, 7, 9), values -> List(42.0, 50000.0, 3.0, 2.0, 1.0))",2
Aaron Chen,57,1,75000,0,4,3,4,5,1,0,"Map(vectorType -> dense, length -> 10, values -> List(57.0, 1.0, 75000.0, 0.0, 4.0, 3.0, 4.0, 5.0, 1.0, 0.0))",3
Aaron Coleman,42,0,50000,0,0,0,3,2,1,1,"Map(vectorType -> dense, length -> 10, values -> List(42.0, 0.0, 50000.0, 0.0, 0.0, 0.0, 3.0, 2.0, 1.0, 1.0))",2


In [0]:
display(classif)

CustomerName,Age,MaritalStatus,IncomeRange,Gender,TotalChildren,ChildrenAtHome,Education,Occupation,HomeOwner,Cars,features,cluster
Aaron Adams,42,0,50000,0,0,0,3,2,1,1,"Map(vectorType -> dense, length -> 10, values -> List(42.0, 0.0, 50000.0, 0.0, 0.0, 0.0, 3.0, 2.0, 1.0, 1.0))",2
Aaron Alexander,40,1,50000,0,0,0,2,2,1,2,"Map(vectorType -> dense, length -> 10, values -> List(40.0, 1.0, 50000.0, 0.0, 0.0, 0.0, 2.0, 2.0, 1.0, 2.0))",2
Aaron Allen,63,0,25000,0,2,1,2,1,1,2,"Map(vectorType -> dense, length -> 10, values -> List(63.0, 0.0, 25000.0, 0.0, 2.0, 1.0, 2.0, 1.0, 1.0, 2.0))",0
Aaron Baker,56,1,50000,0,4,2,2,2,1,2,"Map(vectorType -> dense, length -> 10, values -> List(56.0, 1.0, 50000.0, 0.0, 4.0, 2.0, 2.0, 2.0, 1.0, 2.0))",2
Aaron Bryant,72,0,75000,0,4,0,4,4,1,2,"Map(vectorType -> dense, length -> 10, values -> List(72.0, 0.0, 75000.0, 0.0, 4.0, 0.0, 4.0, 4.0, 1.0, 2.0))",3
Aaron Butler,42,1,75000,0,0,0,3,5,1,2,"Map(vectorType -> dense, length -> 10, values -> List(42.0, 1.0, 75000.0, 0.0, 0.0, 0.0, 3.0, 5.0, 1.0, 2.0))",3
Aaron Campbell,49,0,75000,0,0,0,5,5,1,1,"Map(vectorType -> dense, length -> 10, values -> List(49.0, 0.0, 75000.0, 0.0, 0.0, 0.0, 5.0, 5.0, 1.0, 1.0))",3
Aaron Carter,42,0,50000,0,0,0,3,2,0,1,"Map(vectorType -> sparse, length -> 10, indices -> List(0, 2, 6, 7, 9), values -> List(42.0, 50000.0, 3.0, 2.0, 1.0))",2
Aaron Chen,57,1,75000,0,4,3,4,5,1,0,"Map(vectorType -> dense, length -> 10, values -> List(57.0, 1.0, 75000.0, 0.0, 4.0, 3.0, 4.0, 5.0, 1.0, 0.0))",3
Aaron Coleman,42,0,50000,0,0,0,3,2,1,1,"Map(vectorType -> dense, length -> 10, values -> List(42.0, 0.0, 50000.0, 0.0, 0.0, 0.0, 3.0, 2.0, 1.0, 1.0))",2


In [0]:
classif.select("CustomerName", "cluster").show(10)

+---------------+-------+
 CustomerName|cluster|
+---------------+-------+
 Aaron Adams| 2|
Aaron Alexander| 2|
 Aaron Allen| 0|
 Aaron Baker| 2|
 Aaron Bryant| 3|
 Aaron Butler| 3|
 Aaron Campbell| 3|
 Aaron Carter| 2|
 Aaron Chen| 3|
 Aaron Coleman| 2|
+---------------+-------+
only showing top 10 rows

> You can open Lab05